In [0]:
from pyspark.sql.functions import *

### `Example 1`

In [0]:
data = [("ABC", "Q1", 6000),
       ("XYZ", "Q1", 5000),
       ("ABC", "Q2", 8000),
       ("XYZ", "Q2", 7000)]

schema = ["Company", "Quarter", "Revenue"]

df = spark.createDataFrame(data=data, schema=schema, verifySchema=True)
df.printSchema()
df.display()

root
-- Company: string (nullable = true)
-- Quarter: string (nullable = true)
-- Revenue: long (nullable = true)

Company,Quarter,Revenue
ABC,Q1,6000
XYZ,Q1,5000
ABC,Q2,8000
XYZ,Q2,7000


#### `pivot table`

In [0]:
pivot_df= df.groupBy("Company").pivot("Quarter").agg(sum("Revenue"))
pivot_df.display()

Company,Q1,Q2
XYZ,5000,7000
ABC,6000,8000


#### `unpivot table`

In [0]:
pivot_df.selectExpr("Company", "stack(2, 'Qtr1', Q1, 'Qtr2', Q2) as (Quarter, Revenue)").filter(col("Revenue").isNotNull()).display()

Company,Quarter,Revenue
XYZ,Qtr1,5000
XYZ,Qtr2,7000
ABC,Qtr1,6000
ABC,Qtr2,8000


### `Example 2`

In [0]:
data = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"), \
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"), \
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"), \
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico")]

columns= ["Product","Amount","Country"]

df = spark.createDataFrame(data=data, schema=columns, verifySchema=True)

df.printSchema()

# Show Full Contents of a DataFrame
df.show(truncate=False)

root
-- Product: string (nullable = true)
-- Amount: long (nullable = true)
-- Country: string (nullable = true)

+-------+------+-------+
Product|Amount|Country|
+-------+------+-------+
Banana |1000 |USA |
Carrots|1500 |USA |
Beans |1600 |USA |
Orange |2000 |USA |
Orange |2000 |USA |
Banana |400 |China |
Carrots|1200 |China |
Beans |1500 |China |
Orange |4000 |China |
Banana |2000 |Canada |
Carrots|2000 |Canada |
Beans |2000 |Mexico |
+-------+------+-------+

#### `pivot table`

In [0]:
pivot_df = df.groupBy("Product").pivot("Country").agg(sum("Amount"))
pivot_df.display()

Product,Canada,China,Mexico,USA
Orange,null,4000,null,4000
Beans,null,1500,2000,1600
Banana,2000,400,null,1000
Carrots,2000,1200,null,1500


#### `unpivot table`

In [0]:
pivot_df \
.select("Product", expr("""stack(4, "Canada", Canada, "China", China, "Mexico", Mexico, "USA", USA) as (Counry, Amount)""")) \
.display()

Product,Counry,Amount
Orange,Canada,null
Orange,China,4000
Orange,Mexico,null
Orange,USA,4000
Beans,Canada,null
Beans,China,1500
Beans,Mexico,2000
Beans,USA,1600
Banana,Canada,2000
Banana,China,400


In [0]:
# [row["Product"] for row in df.collect()]

In [0]:
# [row["Amount"] for row in df.collect()]

In [0]:
# [row["Country"] for row in df.collect()]